<a href="https://colab.research.google.com/github/Jeevan85/Ineuron/blob/main/ANN_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!nvidia-smi

Sun Mar 17 05:03:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ROOT = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(ROOT)

In [6]:
!pwd

/content/drive/MyDrive/Colab Notebooks


In [7]:
tf.config.list_logical_devices("GPU")

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
housing = fetch_california_housing()

In [11]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [12]:
housing.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [17]:
import pandas as pd
X= pd.DataFrame(housing.data,columns = housing.feature_names)

In [19]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [21]:
y = pd.DataFrame(housing.target,columns=["target"])

In [22]:
X_train_full,X_test,y_train_full,y_test = train_test_split(X,y,random_state=42)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,random_state=42)

In [24]:
print(X_train_full.shape,X_train.shape,X_valid.shape,y_train_full.shape,y_train.shape,y_valid.shape,X_test.shape,y_test.shape)

(15480, 8) (11610, 8) (3870, 8) (15480, 1) (11610, 1) (3870, 1) (5160, 8) (5160, 1)


In [26]:
LAYERS=[
    tf.keras.layers.Dense(30,activation="relu",input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(10,activation="relu"),
    tf.keras.layers.Dense(5,activation="relu"),
    tf.keras.layers.Dense(1)
]

In [27]:
model=tf.keras.models.Sequential(LAYERS)

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                270       
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
def coeff_determination(y_true,y_pred):
  from keras import backend as k
  SS_res = k.sum(k.square(y_true-y_pred))
  SS_tot = k.sum(k.square(y_true-k.mean(y_true)))
  return (1-SS_res/(SS_tot+k.epsilon()))

In [33]:
model.compile(optimizer="sgd",loss="mse", metrics=[coeff_determination])

In [34]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [37]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_valid,y_valid),verbose=1)

Epoch 1/5
363/363 [==============================] - 3s 8ms/step - loss: 0.3420 - coeff_determination: 0.7248 - val_loss: 0.3198 - val_coeff_determination: 0.7358
Epoch 2/5
363/363 [==============================] - 2s 6ms/step - loss: 0.3357 - coeff_determination: 0.7357 - val_loss: 0.6153 - val_coeff_determination: 0.4345
Epoch 3/5
363/363 [==============================] - 2s 5ms/step - loss: 0.3368 - coeff_determination: 0.7339 - val_loss: 0.3122 - val_coeff_determination: 0.7440
Epoch 4/5
363/363 [==============================] - 1s 4ms/step - loss: 0.3323 - coeff_determination: 0.7376 - val_loss: 0.3145 - val_coeff_determination: 0.7433
Epoch 5/5
363/363 [==============================] - 1s 4ms/step - loss: 0.3280 - coeff_determination: 0.7383 - val_loss: 0.3076 - val_coeff_determination: 0.7459


In [39]:
history =model.fit(X_train,y_train,epochs=5,validation_data=(X_valid,y_valid))

Epoch 1/5
363/363 [==============================] - 1s 4ms/step - loss: 0.3151 - coeff_determination: 0.7468 - val_loss: 0.2955 - val_coeff_determination: 0.7564
Epoch 2/5
363/363 [==============================] - 1s 4ms/step - loss: 0.3119 - coeff_determination: 0.7515 - val_loss: 0.2953 - val_coeff_determination: 0.7573
Epoch 3/5
363/363 [==============================] - 1s 4ms/step - loss: 0.3108 - coeff_determination: 0.7516 - val_loss: 0.2935 - val_coeff_determination: 0.7583
Epoch 4/5
363/363 [==============================] - 1s 4ms/step - loss: 0.3074 - coeff_determination: 0.7502 - val_loss: 0.2931 - val_coeff_determination: 0.7591
Epoch 5/5
363/363 [==============================] - 2s 5ms/step - loss: 0.3064 - coeff_determination: 0.7535 - val_loss: 0.2973 - val_coeff_determination: 0.7581


In [40]:
pd.DataFrame(history.history)

,loss,coeff_determination,val_loss,val_coeff_determination
0,0.315051,0.746823,0.295467,0.756436
1,0.311860,0.751516,0.295319,0.757312
2,0.310833,0.751599,0.293526,0.758331
3,0.307428,0.750242,0.293069,0.759128
4,0.306440,0.753548,0.297287,0.758092


In [41]:
model.evaluate(X_test,y_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.3165 - coeff_determination: 0.7414


[0.31650814414024353, 0.7414168119430542]

In [44]:
X_test[0]

array([-1.15780104, -0.28673138, -0.49550877, -0.16618097, -0.02946012,
        0.38899735,  0.19374821,  0.2870474 ])

In [48]:
(X_test[0].reshape((1,8)))

array([[-1.15780104, -0.28673138, -0.49550877, -0.16618097, -0.02946012,
         0.38899735,  0.19374821,  0.2870474 ]])

In [57]:
X_test[0].shape

(8,)

In [56]:
model.predict(X_test[6].reshape((1,8)))

1/1 [==============================] - 0s 32ms/step


array([[2.4546635]], dtype=float32)